In [ ]:
#All code adapted from: https://www.kaggle.com/code/tard232003/20bd1a6754-basic-shapes , and
# https://www.kaggle.com/code/manuvamshi07/21bd5a6603-shapes-image-handling#Using-ANN
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import image
import os
from sklearn.linear_model import  Lasso, Ridge
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [ ]:
# Input data files are available in the read-only "../input/" directory

for dirname, _, filenames in os.walk('shapes'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
im=image.imread('shapes/squares/drawing(1).png')

In [ ]:
plt.imshow(im)

In [ ]:

#Flattening the images, makes each image into 1d-array -> with the output
from PIL import Image
def load_imgs(impath):
    imgs=[]
    label=[]
    l1=os.listdir(impath)
    for i in l1:
        l2=os.listdir(impath+'/'+i)
        for j in l2:
            c=0
            img=Image.open(impath+i+'/'+j)
            img=img.resize(size=(32,32))
            img=img.convert('L')
            imgs.append(np.array(img).flatten())
            label.append(i)
            del img
    return np.array(imgs),label

In [ ]:
#getting flattened images, each shape( triangle, circle, square) has 100 images, and each image is an array (32 x 32 = 1024 elements), 
# and the output y is the shape each array is associated with.
x,y=load_imgs('shapes/shapes/')
x.shape,len(y)

In [ ]:
# Printing possible truth - labels
target=pd.Series(y,dtype='category')
target

In [ ]:
#getting codes of truths
t=target.cat.codes
t

In [ ]:
#splitting data into test and training
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
train_x,test_x,train_y,test_y=train_test_split(x,t,test_size=0.2,shuffle=True)
train_x.shape,test_x.shape,train_y.shape,test_y.shape

In [192]:
#LOGISTIC REGRESSION model
from sklearn.linear_model import LogisticRegression
lg=LogisticRegression(max_iter=200,solver='saga')
lg.fit(train_x,train_y)

c:\Users\Student\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=200, solver='saga')

In [193]:
yhat=lg.predict(test_x)   #making predictions, but modle might need more training, has no validation phase
yhat

array([0, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 2, 2, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 2, 2, 1, 0, 1, 1, 2, 0, 2, 0, 0, 1, 2, 2, 1, 1, 2, 0, 0, 1,
       1, 1, 1, 2, 2, 1, 0, 2, 2, 2, 0, 1, 0, 0, 0, 1], dtype=int8)

In [194]:
#mse
mean_squared_error(test_y,yhat)

0.7833333333333333

In [189]:
#load SVC model and training
from sklearn.svm import SVC 
lg=SVC()
lg.fit(train_x,train_y)

SVC()

In [190]:
#prediction
yhat=lg.predict(test_x)
yhat


array([0, 0, 0, 1, 0, 1, 2, 1, 1, 0, 1, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 2, 0, 1, 0, 1, 1, 2, 1, 2, 0, 1, 2, 2, 2, 1, 1, 2, 1, 0, 1,
       2, 2, 0, 2, 2, 1, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0], dtype=int8)

In [191]:
 #computing MSE
mean_squared_error(test_y,yhat)

0.5833333333333334

In [ ]:
# Define linear models with regularization
l1_reg = Lasso(alpha=0.1, max_iter=10000, normalize=True)
l2_reg = Ridge(alpha=0.1, max_iter=10000, normalize=True)

In [ ]:
#Train with same data
l1_reg.fit(train_x,train_y)
l2_reg.fit(train_x,train_y)

In [ ]:
y_hat_l1 = l1_reg.predict(test_x)
y_hat_l2 = l2_reg.predict(test_x)


In [ ]:
#MSE
l1 = mean_squared_error(test_y, y_hat_l1)
l1

In [ ]:
#MSE
l2 = mean_squared_error(test_y, y_hat_l1)
l2

## Using the Holdout Method

In [167]:
#Splittin data in into training, validation and prediction
X_learn, X_test, y_learn, y_test = train_test_split(x, t, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_learn, y_learn, test_size=0.2, random_state=1)

In [168]:
#models
lin_reg = make_pipeline(MinMaxScaler(), LogisticRegression(max_iter=200,solver='saga'))
l1_reg = make_pipeline(MinMaxScaler(), Lasso(alpha=0.1, max_iter=10000))
l2_reg = make_pipeline(MinMaxScaler(),Ridge(alpha=0.1, max_iter=10000))
sv = make_pipeline(MinMaxScaler(),SVC())

In [169]:
#Training
lin_reg.fit(X_train, y_train)
l1_reg.fit(X_train, y_train)
l2_reg.fit(X_train, y_train)
sv.fit(X_train, y_train)

c:\Users\Student\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('svc', SVC())])

In [170]:
#Validate
lin_reg.fit(X_val, y_val)
l1_reg.fit(X_val, y_val)
l2_reg.fit(X_val, y_val)
sv.fit(X_val, y_val)

c:\Users\Student\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('svc', SVC())])

In [171]:
#Test
y1=lin_reg.predict(X_test)
y2=l1_reg.predict(X_test)
y3=l2_reg.predict(X_test)
y4=sv.predict(X_test)

In [172]:
#MSE of logitic REgression after holdout
mean_squared_error(y_test, y1)

0.8666666666666667

In [173]:
#MSE of lasso model after holdout
mean_squared_error(y_test, y2)

0.6657118055555554

In [174]:
#MSE of Ridge model after holdout
mean_squared_error(y_test, y3)

0.72954762815902

In [175]:
#MSE of lSVC model
mean_squared_error(y_test, y4)

1.35

## 3 Way Holdout

In [176]:
def threeWayHoldout(X, y, test_size=0.5, val_size=0.1, models=make_pipeline(MinMaxScaler(),SVC())):
    score_dict = {}
    X_learn, X_test, y_learn, y_test = train_test_split(X, y, test_size=test_size, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_learn, y_learn, test_size=val_size, random_state=1)
        
    alphas = np.logspace(-8, 1, num=10)
    for alpha in alphas:
        reg = models
        reg.fit(X_train, y_train)
        score_dict[alpha] = reg.score(X_val, y_val)
        
    return score_dict, X_learn, X_test, y_learn, y_test

In [177]:
#2 Way HoldOut will be used to only optimise Lasso and Ridge
l1_reg = make_pipeline(MinMaxScaler(), Lasso(alpha=0.1, max_iter=10000))
l2_reg = make_pipeline(MinMaxScaler(),Ridge(alpha=0.1, max_iter=10000))

In [178]:
score_dict, X_learn, X_test, y_learn, y_test  = threeWayHoldout(x, t, test_size=4.0/5.0, val_size=0.3, models=l1_reg)
score_dict_ridge, X_learn_ridge, X_test_ridge, y_learn_ridge, y_test_ridge  = threeWayHoldout(x, t, test_size=4.0/5.0, val_size=0.3, models=l2_reg)

In [164]:
for alpha, score in score_dict.items():
    print("Alpha: {}\tScore: {}".format(alpha, score))

Alpha: 1e-08	Score: -0.026313582428547422
Alpha: 1e-07	Score: -0.026313582428547422
Alpha: 1e-06	Score: -0.026313582428547422
Alpha: 1e-05	Score: -0.026313582428547422
Alpha: 0.0001	Score: -0.026313582428547422
Alpha: 0.001	Score: -0.026313582428547422
Alpha: 0.01	Score: -0.026313582428547422
Alpha: 0.1	Score: -0.026313582428547422
Alpha: 1.0	Score: -0.026313582428547422
Alpha: 10.0	Score: -0.026313582428547422


In [179]:
for alpha, score in score_dict_ridge.items():
    print("Alpha: {}\tScore: {}".format(alpha, score))

Alpha: 1e-08	Score: -0.27311966137416466
Alpha: 1e-07	Score: -0.27311966137416466
Alpha: 1e-06	Score: -0.27311966137416466
Alpha: 1e-05	Score: -0.27311966137416466
Alpha: 0.0001	Score: -0.27311966137416466
Alpha: 0.001	Score: -0.27311966137416466
Alpha: 0.01	Score: -0.27311966137416466
Alpha: 0.1	Score: -0.27311966137416466
Alpha: 1.0	Score: -0.27311966137416466
Alpha: 10.0	Score: -0.27311966137416466


In [183]:
alpha = max(score_dict, key=score_dict.get)
l1_reg =make_pipeline(MinMaxScaler(), Lasso(alpha=alpha, max_iter=10000))
l1_reg.fit(X_learn, y_learn)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('lasso', Lasso(alpha=1e-08, max_iter=10000))])

In [184]:
alpha = max(score_dict, key=score_dict.get)
l2_reg = make_pipeline(MinMaxScaler(), Ridge(alpha=alpha, max_iter=10000))
l2_reg.fit(X_learn_ridge, y_learn_ridge)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('ridge', Ridge(alpha=1e-08, max_iter=10000))])

In [185]:
y1 = l1_reg.predict(X_test)
y2 = l2_reg.predict(X_test_ridge)

In [186]:
#Lasso, perfromed worse 
mean_squared_error(y_test, y1)

6.71037809267195

In [187]:
#Ridge performed worse
mean_squared_error(y_test, y2)

1.0551693165831215